In [22]:
from langchain_openai import OpenAIEmbeddings
from graphdatascience import GraphDataScience
from getpass import getpass
import pandas as pd
import os
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from IPython.display import display
import string

# Set up connection to Neo4j

In [23]:
neo4j_password = "diameter-captures-caves"

In [24]:
neo4j_database_name = "neo4j"
neo4j_user = "neo4j"
bolt_url = "bolt://52.87.157.164:7687"
gds = GraphDataScience(bolt_url, auth=(neo4j_user, neo4j_password))

# Find themes with common stems

In [25]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
theme_df = gds.run_cypher("""
    MATCH (t:Theme) 
    RETURN toLower(t.description) AS description 
    """)

In [27]:
len(theme_df)

53591

In [28]:
theme_df.head()

,description
0,- return only the list without a heading
1,- return the list as a pipe-separated list
2,12) if a theme is a commentary on society
3,13) if the movie has won any awards
4,13) the list should not exceed eight elements


In [29]:
theme_df['stem'] = theme_df['description'].map(lambda x: lemmatizer.lemmatize(x))

In [30]:
table = str.maketrans("", "", string.punctuation)

In [31]:
theme_df['no_space_stem'] = theme_df['stem'].str.replace(" ", "")
theme_df['no_space_stem'] = theme_df['no_space_stem'].str.translate(table)

In [32]:
theme_df[theme_df['description'] == 'haunted house']

,description,stem,no_space_stem
20249,haunted house,haunted house,hauntedhouse
20258,haunted house,haunted house,hauntedhouse


In [33]:
grouped_themes = theme_df[['description', 'no_space_stem']].groupby('no_space_stem').apply(np.unique, include_groups=False)

In [34]:
shared_stems = grouped_themes[grouped_themes.map(len)>1]
shared_stems_df = shared_stems.reset_index()
shared_stems_df.columns = ['stem', 'descriptions']
shared_stems_df.shape

(2075, 2)

In [35]:
shared_stems_df.sample(10)

,stem,descriptions
430,crater,"[crater, craters]"
1219,motherland,"[- motherland, motherland]"
1521,regular,"[regular, regulars]"
1479,qualification,"[qualification, qualifications]"
594,ear,"[ear, ears]"
37,act,"[act, acts]"
1225,movie,"['movie', movie, movies]"
1485,rabbithole,"[rabbit hole, rabbit-hole]"
326,city,"[cities, city]"
1811,surveillance,"[- surveillance, surveillance]"


## Send stems shared by multiple themes to Neo4j

In [36]:
gds.run_cypher("""CREATE CONSTRAINT stem_node_key IF NOT EXISTS FOR (s:Stem) REQUIRE s.stem IS NODE KEY""")

""


In [37]:
gds.run_cypher("""UNWIND $data AS row
MERGE (s:Stem {stem:row['stem']})
WITH row, s
UNWIND row['descriptions'] AS description
MATCH (t:Theme)
WHERE toLower(t.description) = description
MERGE (t)-[:HAS_STEM]->(s)
SET s.descriptions = row['descriptions']
RETURN count(*) AS stemmedCount""",
              {"data": shared_stems_df.to_dict("records")})

,stemmedCount
0,5113


## Aggregate theme vectors for themes that share a stem

Set the stem's vector to whichever theme shows up most often in movies

In [38]:
gds.run_cypher("""
MATCH (s:Stem)
CALL {
WITH s
MATCH (s)<-[:HAS_STEM]-(t)
WITH s, t
ORDER BY count{(t)<-[:HAS_THEME]-()} DESC
WITH s, collect(t) AS themes
CALL db.create.setNodeVectorProperty(s, "embedding", themes[0].embedding)
}
""")

""


In [39]:
shared_stem_df = gds.run_cypher("""
MATCH (s:Stem)<-[:HAS_STEM]-(t)
RETURN s.stem AS stem, collect(t.embedding) AS embeddings
""")

In [40]:
def get_mean_vector(row):
    mean_vector = np.array(row['embeddings']).mean(axis=0).tolist()
    return mean_vector

In [41]:
shared_stem_df['mean_embedding'] = shared_stem_df.apply(get_mean_vector, axis=1)

In [42]:
gds.run_cypher("""
    UNWIND $data AS row
    MATCH (s:Stem {stem: row['stem']})
    CALL db.create.setNodeVectorProperty(s, "embedding", row['mean_embedding'])""",
                   {"data":shared_stem_df[['stem', 'mean_embedding']].to_dict("records")})


""


# Create projection
Include stems and individual themes that do not share a stem with another theme.

Use the theme embedding and the aggregated embedding for stems.

In [43]:
gds.run_cypher("""
MATCH (t:Stem|Theme)
WHERE NOT EXISTS {(t)-[:HAS_STEM]->()}
SET t:Groupable""")

""


In [52]:
gds.run_cypher("""CREATE VECTOR INDEX groupbale_vectors IF NOT EXISTS 
                  FOR (t:Groupable)
                  ON (t.embedding)
                  OPTIONS {indexConfig: 
                      {`vector.dimensions`: 768,
                       `vector.similarity_function`: 'cosine'
                       }}
                       """)

""


In [54]:
g_themes, result = gds.graph.cypher.project("""
MATCH (t:Groupable)
RETURN gds.graph.project("themes", 
t, null,
{sourceNodeProperties: t{.embedding},
targetNodeProperties: null}
)""")

In [55]:
result

{'relationshipCount': 0,
 'graphName': 'themes',
 'query': '\nMATCH (t:Groupable)\nRETURN gds.graph.project("themes", \nt, null,\n{sourceNodeProperties: t{.embedding},\ntargetNodeProperties: null}\n)',
 'projectMillis': 1424,
 'configuration': {'readConcurrency': 4,
  'undirectedRelationshipTypes': [],
  'jobId': '0af42097-81a8-40a4-82ba-c161ea520d8c',
  'logProgress': True,
  'query': '\nMATCH (t:Groupable)\nRETURN gds.graph.project("themes", \nt, null,\n{sourceNodeProperties: t{.embedding},\ntargetNodeProperties: null}\n)',
  'inverseIndexedRelationshipTypes': [],
  'creationTime': neo4j.time.DateTime(2024, 7, 9, 4, 34, 40, 780628453, tzinfo=<UTC>)},
 'nodeCount': 50553}

# Test some theme pairs to find a score above which you think themes are probably duplicated
Get a sense of the range of similarity values for top 5 nearest neighbors.

Note that KNN returns cosine similarity normalized to the range [0, 1]. 

If you run gds.similarity.cosine() you get similarity in the range [-1,1]

In [56]:
knn_stats = gds.knn.stats(g_themes, topK=3, nodeProperties = "embedding")

Knn:   0%|          | 0/100 [00:00<?, ?%/s]

In [57]:
knn_stats['similarityDistribution']

{'min': 0.5918083190917969,
 'p5': 0.6946563720703125,
 'max': 1.0000038146972656,
 'p99': 0.9852828979492188,
 'p1': 0.6640357971191406,
 'p10': 0.7116203308105469,
 'p90': 0.8766365051269531,
 'p50': 0.7810211181640625,
 'p25': 0.7418403625488281,
 'p75': 0.8284759521484375,
 'p95': 0.9070777893066406,
 'mean': 0.7886515077779553,
 'p100': 1.0000038146972656,
 'stdDev': 0.06549347188129939}

## Given a theme, check to see how many neighbors are duplicates and the similarity score for the first neighbor you would consider to be a different theme.

db.index.vector.queryNodes() returns cosine similarity normalized to the range [0, 1]

In [58]:
top_k_df = gds.run_cypher("""
MATCH (t1:Groupable) WHERE t1.description IN $themeList OR ANY(str IN $themeList WHERE str IN t1.descriptions)
CALL db.index.vector.queryNodes("groupbale_vectors", $k + 1, t1.embedding) YIELD node, score
WITH t1, node AS t2, score
WHERE t1 <> t2
RETURN coalesce(t1.description, reduce(str=t1.descriptions[0], x in t1.descriptions[1..] | str + ", " + x)) AS theme1,
coalesce(t2.description, reduce(str=t2.descriptions[0], x in t2.descriptions[1..] | str + ", " + x)) AS theme2,
score
ORDER BY theme1, score DESC""",
               {"k": 20,
                "themeList": ['friend', 'dance floor', 'fast food', 'Influential artist', 'antics', 
                              'ants', 'underwater', 'Africa', 'chatgpt', 'coming-of-age']})

In [59]:
top_k_df['theme1'].unique()

array(['Africa', 'chat-gpt, chatgpt', 'coming of age, coming-of-age',
       'friend, friends'], dtype=object)

In [60]:
top_k_df[top_k_df['theme1'] == 'friend, friends']
# top 2 cutoff 0.825685

,theme1,theme2,score
60,"friend, friends","""friendship"", - friendship, friendship, friend...",0.912966
61,"friend, friends",Close friends,0.911946
62,"friend, friends",Close Friends,0.911946
63,"friend, friends","best friends, bestfriends",0.890089
64,"friend, friends",Befriends,0.881329
65,"friend, friends",Longtime Friends,0.880000
66,"friend, friends","- good friends, good friends",0.868967
67,"friend, friends",Frenemies,0.867264
68,"friend, friends",Childhood friends,0.866182
69,"friend, friends",Childhood Friends,0.866182


In [61]:
top_k_df[top_k_df['theme1'] == 'dance floor, dancefloor']
# top 0 cutoff 0.871256

,theme1,theme2,score


In [62]:
top_k_df[top_k_df['theme1'] == 'fast food']
# top 2 cutoff 0.804882

,theme1,theme2,score


In [63]:
top_k_df[top_k_df['theme1'] == 'Influential artist']
# top 9 cutoff 0.787514

,theme1,theme2,score


In [64]:
top_k_df[top_k_df['theme1'] == 'antics']
# top 0 cutoff 0.854217

,theme1,theme2,score


In [65]:
top_k_df[top_k_df['theme1'] == 'ants']
# top 0 cutoff 0.854217

,theme1,theme2,score


In [66]:
top_k_df[top_k_df['theme1'] == 'underwater']
# top 4 cutoff 0.880904

,theme1,theme2,score


In [67]:
top_k_df[top_k_df['theme1'] == 'Africa']
# top 0 cutoff 0.829958

,theme1,theme2,score
0,Africa,West Africa,0.922315
1,Africa,North African,0.887995
2,Africa,Tanzania,0.884488
3,Africa,"- african, african",0.881666
4,Africa,West African,0.881114
5,Africa,Angola,0.874282
6,Africa,African setting,0.872613
7,Africa,Congo,0.867494
8,Africa,South Africa,0.864438
9,Africa,Uganda,0.860408


In [68]:
top_k_df[top_k_df['theme1'] == 'chat-gpt, chatgpt']
# top 1 cutoff 0.725718

,theme1,theme2,score
20,"chat-gpt, chatgpt",Chat,0.826902
21,"chat-gpt, chatgpt",Sexchat,0.818864
22,"chat-gpt, chatgpt",Chatworld,0.816521
23,"chat-gpt, chatgpt",Secret Chat,0.810010
24,"chat-gpt, chatgpt",Private Chat,0.802077
25,"chat-gpt, chatgpt",Video Chat,0.798567
26,"chat-gpt, chatgpt",Chatting,0.779387
27,"chat-gpt, chatgpt",Discord,0.777126
28,"chat-gpt, chatgpt",discord,0.777126
29,"chat-gpt, chatgpt",Chattopadhyaya,0.776135


In [69]:
top_k_df[top_k_df['theme1'] == 'coming of age, coming-of-age']
# top 3 cutoff 0.792982

,theme1,theme2,score
40,"coming of age, coming-of-age",Coming-of-Middle-Age,0.932474
41,"coming of age, coming-of-age",Queer Coming-of-Age,0.898571
42,"coming of age, coming-of-age",Growing Up,0.858810
43,"coming of age, coming-of-age",Growing up,0.858810
44,"coming of age, coming-of-age",growing up,0.858810
45,"coming of age, coming-of-age",Growing Old,0.857799
46,"coming of age, coming-of-age",Growing old,0.857799
47,"coming of age, coming-of-age",Maturing,0.856477
48,"coming of age, coming-of-age",Young Adulthood,0.848055
49,"coming of age, coming-of-age",Teenagehood,0.824367


# Based on the exploration above run KNN with a similarityCutoff you choose

In [70]:
similarity_cutoff = 0.83

knn_result = gds.knn.mutate(g_themes,
                            mutateRelationshipType = "IS_SIMILAR",
                            mutateProperty = "similarity",
                            nodeProperties = "embedding",
                            topK = 2,
                            sampleRate = 1.0,
                            perturbationRate = 0.5,
                            deltaThreshold = 0.00001,
                            similarityCutoff = similarity_cutoff)

Knn:   0%|          | 0/100 [00:00<?, ?%/s]

## Find themes that discovered no neighbors above the similarity threshold you selected

In [71]:
no_sim_df = gds.run_cypher("""
CALL gds.graph.relationships.stream("themes", ["IS_SIMILAR"]) yield sourceNodeId, targetNodeId
UNWIND [gds.util.asNode(sourceNodeId), gds.util.asNode(targetNodeId)] AS nodeWithSim
WITH collect(DISTINCT nodeWithSim) AS nodesWithSim
MATCH (t:Theme) WHERE NOT t IN nodesWithSim
AND NOT EXISTS {(t)-[:HAS_STEM]->()}
RETURN t.description AS description, count{ (t)<-[:HAS_THEME]-() } AS movieCount
ORDER BY movieCount desc""")

In [72]:
no_sim_df

,description,movieCount
0,Man,226
1,**Instruction 2 (More Difficult):**,88
2,Justice,82
3,Paris,63
4,Hollywood,60
...,...,...
34342,Time-Travel Sequences,0
34343,Time-less Charm,0
34344,Timekeeper,0
34345,Tiny Act,0


In [73]:
no_sim_df[no_sim_df['movieCount'] > 1]['description'].count()

4429

## Examine theme pairs for high degree nodes
For Stem nodes, turn their list of descriptions into a comma separated string before exporting.

In [74]:
similarity_result = gds.run_cypher("""
    CALL gds.graph.relationshipProperty.stream("themes", "similarity", ["IS_SIMILAR"])
    YIELD sourceNodeId, targetNodeId, propertyValue AS similarity
    WITH gds.util.asNode(sourceNodeId) AS source, gds.util.asNode(targetNodeId) AS target, similarity
    RETURN coalesce(source.description, reduce(s = source.descriptions[0], d in source.descriptions[1..] | s + ", " + d)) AS source, 
    coalesce(target.description, reduce(s = target.descriptions[0], d in target.descriptions[1..] | s + ", " + d)) AS target, similarity
    ORDER BY similarity""")
                    

What are the nodes with the most inbound IS_SIMILAR relationships?

In [75]:
similarity_result.groupby("target")['source'].count().sort_values(ascending=False).head(20)

target
Artistic Creation                                                                                                                                                                                            39
Life\n\n\n\nYou are a film critic with a focus on cultural impact.\n    You are given the title and a brief synopsis of a movie.\n    Extract and list the most significant themes                           30
- filmmaking, film making, filmmaking                                                                                                                                                                        26
artistic creation                                                                                                                                                                                            20
You are a film critic with a focus on character development and narrative structure.\n\n    You are given the title and a detailed synopsis of a movie.\n    Extr

In [76]:
similarity_result[similarity_result['target']=="haunted house"]

,source,target,similarity


In [77]:
similarity_result[similarity_result['target']=="hidden secrets"]

,source,target,similarity


In [78]:
similarity_result[similarity_result['target']=="Christmas magic"]

,source,target,similarity


In [79]:
similarity_result[similarity_result['target']=="documentary video"]

,source,target,similarity


# Write weakly connected components with threshold at your KNN similarityCutoff or a higher value
I'm bumping the threshold up to 0.875

In [80]:
gds.wcc.write(g_themes,
              writeProperty = "wccId",
              relationshipTypes = ["IS_SIMILAR"],
              relationshipWeightProperty = "similarity",
              threshold = 0.875,
              minComponentSize=2)

writeMillis                                                            182
nodePropertiesWritten                                                 6464
componentCount                                                       46272
componentDistribution    {'min': 1, 'p5': 1, 'max': 85, 'p999': 10, 'p9...
postProcessingMillis                                                    27
preProcessingMillis                                                      0
computeMillis                                                           34
configuration            {'writeProperty': 'wccId', 'jobId': 'adf710ad-...
Name: 0, dtype: object

## Examine the largest communities at the KNN value you selected

In [81]:
pd.set_option('display.max_colwidth', None)

In [82]:
gds.run_cypher("""
MATCH (t:Theme)
WHERE t.wccId IS NOT NULL
WITH t.wccId AS id, collect(t.description) AS themes
RETURN id, size(themes) AS themeCount, themes
ORDER BY themeCount DESC
LIMIT 30""")

id  themeCount  \
0     604          85   
1    3265          82   
2     185          76   
3    2515          43   
4     132          38   
5    1416          33   
6    3333          30   
7      77          30   
8    4343          22   
9    2584          20   
10    515          20   
11    259          18   
12   1515          17   
13   1306          16   
14   8478          16   
15  13285          16   
16   7632          15   
17   1340          15   
18     48          14   
19    449          13   
20   5485          13   
21    221          13   
22   8363          12   
23   3846          12   
24    657          12   
25   3148          12   
26   4832          12   
27    522          12   
28   3256          12   
29   2435          12   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

# Create undirected graph for Leiden

The largest WCC communities are too big. For example, "enchanted forest" and "buried body" do not belog in the same community. We'll use Leiden to make smaller comminities.

Leiden requires undirected relationships, so convert the directed IS_SIMILAR relationship to an undirected relationship called "UNDIRECTED_SIMILAR".

In [83]:
gds.run_cypher("""
call gds.graph.relationships.toUndirected("themes", {relationshipType: "IS_SIMILAR", mutateRelationshipType: "UNDIRECTED_SIMILAR", aggregation: "MAX"})""")

,inputRelationships,relationshipsWritten,mutateMillis,postProcessingMillis,preProcessingMillis,computeMillis,configuration
0,22646,28634,0,0,0,68,"{'aggregation': 'MAX', 'relationshipType': 'IS_SIMILAR', 'jobId': '4f2b1447-4c3a-43fe-81b8-e240fff71592', 'logProgress': True, 'concurrency': 4, 'sudo': False, 'mutateRelationshipType': 'UNDIRECTED_SIMILAR'}"


## Transform relationship weights
Because of our similarity cutoff in KNN, all of our relationship scores are between 0.811518 and 1.0. Apply a transformation to spread them out between 0.0 and 1.0.

In [84]:
gds.set_database("neo4j")

In [85]:
gds.run_cypher("""
CALL gds.graph.relationshipProperties.stream($graphName, ["similarity"], ["UNDIRECTED_SIMILAR"])
YIELD sourceNodeId, targetNodeId, propertyValue
WITH sourceNodeId AS source, 
targetNodeId AS target,
max(propertyValue) AS score
WITH gds.graph.project("reweight_themes",
source,
target,
{relationshipType: "IS_SIMILAR",
relationshipProperties: {similarity: (score-$similarityCutoff)/(1-$similarityCutoff)}},
{undirectedRelationshipTypes: ["IS_SIMILAR"]}) AS g
RETURN g.graphName AS graphName, g.relationshipCount AS relationshipCount, g.nodeCount AS nodeCount""",
              {"graphName": g_themes.name(), "similarityCutoff": similarity_cutoff})

,graphName,relationshipCount,nodeCount
0,reweight_themes,57268,14998


In [86]:
g_themes2 = gds.graph.get("reweight_themes")

# Test Leiden with different values of gamma
This function will run the Leiden algorithm with a value of gamma that you provide. It will return the size of the biggest Leiden communities and the themes in those communities. It will also check for the smallest communities that contain only one document.

In [87]:
def test_leiden_gamma(gamma):
    property_name = f"leidenGamma{gamma}"
    try:
        gds.graph.nodeProperties.drop(g_themes2, property_name)
    except:
        pass
    leiden_result = gds.leiden.mutate(g_themes2,
                      mutateProperty = property_name,
                      relationshipTypes = ["IS_SIMILAR"],
                      relationshipWeightProperty = "similarity",
                      gamma=gamma)
    biggest_communities = gds.run_cypher("""
        CALL gds.graph.nodeProperty.stream("reweight_themes", $propName)
        YIELD nodeId, propertyValue
        WITH propertyValue, collect(gds.util.asNode(nodeId)) AS nodeList, count(*) as themeCount
        ORDER BY themeCount desc
        LIMIT 20
        RETURN themeCount, [n in nodeList | coalesce(n.description, n.descriptions)] AS themes""",
                                        {"propName": property_name})
    stats_df = gds.run_cypher("""
        CALL gds.graph.nodeProperty.stream("reweight_themes", "leidenGamma1.0")
        YIELD nodeId, propertyValue
        WITH gds.util.asNode(nodeId) as n, propertyValue AS leidenId
        WITH leidenId, collect(n) AS nodeList
        WHERE size(nodeList) = 1
        WITH leidenId, nodeList[0] AS n
        WITH leidenId,
        COUNT{MATCH (n)<-[:HAS_THEME]-(d) RETURN DISTINCT d} AS themeCount,
        COUNT{MATCH (n)<-[:HAS_STEM]-()<-[:HAS_THEME]-(d) RETURN DISTINCT d} AS stemCount
        WHERE themeCount + stemCount = 1
        RETURN count(*) AS singleDocCommunities""",
                                           {"propName": property_name})
    stats_df.loc[0,'community_count'] = leiden_result['communityCount']
    stats_df.loc[0, 'max_community_size'] = leiden_result['communityDistribution']['max']
    stats_df.loc[0, 'mean_community_size'] = leiden_result['communityDistribution']['mean']
    stats_df.loc[0, 'median_community_size'] = leiden_result['communityDistribution']['p50']
    stats_df.loc[0, 'gamma'] = gamma
    return {"biggest_communities": biggest_communities, "stats": stats_df}
                                           


In [88]:
gamma_tests = {gamma: test_leiden_gamma(gamma) for gamma in [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0, 1024.0]}

Leiden:   0%|          | 0/100 [00:00<?, ?%/s]

In [89]:
stats_df = pd.concat([r['stats'] for r in [gamma_tests[key] for key in gamma_tests.keys()]])
stats_df[['gamma', 'singleDocCommunities', 'community_count', 'max_community_size', 'mean_community_size', 'median_community_size']]

,gamma,singleDocCommunities,community_count,max_community_size,mean_community_size,median_community_size
0,1.0,0,3515.0,197.0,4.266856,2.0
0,2.0,0,3526.0,197.0,4.253545,2.0
0,4.0,0,3553.0,185.0,4.221222,2.0
0,8.0,0,3594.0,112.0,4.173066,2.0
0,16.0,0,3651.0,85.0,4.107916,2.0
0,32.0,0,3724.0,68.0,4.027390,2.0
0,64.0,0,3845.0,45.0,3.900650,2.0
0,128.0,0,4024.0,32.0,3.727137,2.0
0,256.0,0,4258.0,21.0,3.522311,2.0
0,512.0,0,4620.0,18.0,3.246320,2.0


In [90]:
for gamma, result in gamma_tests.items():
    print(f"Gamma value: {gamma}")
    display(result['biggest_communities'].head())

Gamma value: 1.0


themeCount  \
0         197   
1         175   
2         147   
3         142   
4         123   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

Gamma value: 2.0


themeCount  \
0         197   
1         142   
2         134   
3          95   
4          92   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

Gamma value: 4.0


...

,themeCount,themes
0,21,"[Cultural, Cultural Innovation, cultural activity, American culture, Socio-Cultural, Cultural Perspectives, Cultural setting, Cultural Roots, [culture, cultures], Cultural Life, Intercultural, Cultural acceptance, Cultural Influence, Funerary Culture, Vibrant cultures, Chinese culture, cultural critique, Image Culture, Hip-Hop Culture, third culture, [subculture, subcultures]]"
1,21,"[Syed Abdul Rahim, Amar Singh Chamkila, Iqbal Singh, Naeem Tahir, Amar Singh, Ashan Singh, Akaash Singh, Singh, Hari Singh, Alok Patel, Abhimanyu Mukherjee, Ashwini Puneeth Rajkumar, Azaad Singh, Syed Shakil, Dhar Mukherjee, Bahadur Khan, Aadavan, Sourabh Sharma, Satya Harishandra Prasad, Hind Rajab, Raghuram]"
2,21,"[Karina Chikitova, Chrysoula Rodaki, Irina Viner Usmanova, Lisandra Chirinos, Maryana Dvorska, Lyudmila Kusakova, Sofia Cerasoli, Paula Ďurinová, Margarita Denysova, Maria Leiko, Mari Järsk, Sonia Oleniak, Alma-Sofia Miettinen, Alexandra Kollontaï, Tatyana Navka, Marina Tsvetaeva, Ivanna Tsareva, Elena Sokolova. Set against the backdrop of the tumultuous 1960s in the Soviet Union, Ioulia Stavridou, Anna Maria Kakona, Beata Parkanová]"
3,21,"[Short Stories, Short Films, Self-made Films, [- film, film, films], Shanghai International Film Festival, Singapore Cinema, Film class, Home movies, Silent Films, Film History, film-within-film, Hit Films, Reality in Cinema, ['cinema', cinema], Unmade Films, Film Competition, three films, Silent Film, The director's influence on the film industry, film existence, 6) Avoid using the words 'film']"
4,19,"[[dream, dreams], Dream Revelation, Big Dreams, Dreaming, Dream World, Coffee Dream, Sex Dream, Mass Dreaming, Artist dream, Dream #1, Dream-reality, Deadly dreams, dream figment, One Night Dream, Dream world, Dream Living, i dreamed, Idol dream, [nightmare, nightmares]]"


Gamma value: 512.0


,themeCount,themes
0,18,"[Self-made Films, [- film, film, films], Shanghai International Film Festival, Singapore Cinema, Film class, Silent Films, Film History, film-within-film, Hit Films, Reality in Cinema, ['cinema', cinema], Unmade Films, Film Competition, three films, Silent Film, The director's influence on the film industry, film existence, 6) Avoid using the words 'film']"
1,16,"[Rita Lopez, María García Ruiz, Norma Reyes, Dora, Dora García, Vera Miró, Mariana Medina, Julia Vera, Virgina García del Pino, Mariela Guerrero Monegro, Tavera Pena Chabelli, Dyessa Garcia, Dana Rodriguez, Viviana Gallardo, Mariana Torta, Alisa Guiamoy]"
2,15,"[Family Life, Family unit, [- family, families, family], Family Lives, Couple life, Family Approval, Family Drama, New Family, Family Loyalty, Divided Family, Extended Family, Gay Family, Alien family, Families & Faces, Family Dignity]"
3,14,"[Best Friend, [friend, friends], Four Friends, Freak Friendship, Close Friends, Frenemies, BFFs, Besties, Few friends, College friends, Six Friends, Friend A, BFF, New friends]"
4,14,"[[dream, dreams], Dream Revelation, Big Dreams, Dreaming, Coffee Dream, Sex Dream, Mass Dreaming, Artist dream, Dream #1, Dream-reality, One Night Dream, Dream Living, i dreamed, Idol dream]"


Gamma value: 1024.0


,themeCount,themes
0,12,"[Family unit, [- family, families, family], Family Approval, Family Drama, New Family, Family Loyalty, Divided Family, Extended Family, Gay Family, Alien family, Families & Faces, Family Dignity]"
1,12,"[Hikaru, Uyama Hiroto, Yoma Yutaka, Yumi Aihara, Yuima Nakazato, Sheina Yu, Yuki Ueno, Yuka Higuchi, Daisuke Sasaki, Yuzuki Aikawa, Riko Fukumoto, Yuzuki Victoria]"
2,12,"[[dream, dreams], Dream Revelation, Big Dreams, Dreaming, Coffee Dream, Mass Dreaming, Artist dream, Dream #1, Dream-reality, Dream Living, i dreamed, Idol dream]"
3,11,"[American culture, Cultural Perspectives, [culture, cultures], Cultural acceptance, Cultural Influence, Chinese culture, cultural critique, Image Culture, Hip-Hop Culture, third culture, [subculture, subcultures]]"
4,10,"[Street Cat, [cat, cats], Wild Cats, [feline, felines], Captive Cats, Nature Cat, Lost Cats, Catspiracy, Community cats, Cat Mouse]"


## For individual themes, see what other themes drop out of their communities as you increase gamma

In [91]:
def get_proprety_df(property_name):
    property_df = gds.graph.nodeProperties.stream(g_themes2,
                                                  [property_name],
                                                  db_node_properties = ["description", "descriptions"],
                                                  separate_property_columns=True)
    property_df.loc[pd.notna(property_df['descriptions']), 'descriptions'] = property_df.loc[pd.notna(property_df['descriptions']), 'descriptions'].map(lambda x: ", ".join(x))
    property_df['description'] = property_df.apply(lambda row: row['description'] if row['description'] else row['descriptions'], axis=1)
    property_df.drop('descriptions', axis=1, inplace=True)
    return property_df

property_dfs = {property_name: get_proprety_df(property_name)
                for property_name in [f"leidenGamma{gamma}" for gamma in [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0]]}

In [92]:
def get_cluster_for_theme(property_name, theme):
    theme_df = property_dfs[property_name]
    result_list = []
    if theme in theme_df['description'].to_list():
        theme_cluster_id = theme_df.loc[theme_df['description']==theme, property_name].iloc[0]
        result_list = theme_df.loc[theme_df[property_name]==theme_cluster_id, "description"].to_list()
    return result_list

In [93]:
def compare_gamma_for_theme(theme, gammas):
    column_names = [f"leidenGamma{gamma}" for gamma in gammas]
    lists = [get_cluster_for_theme(column, theme) for column in column_names]
    output_df = pd.DataFrame()
    output_df.loc[column_names[0], "final_themes"] = ", ".join(lists[0])
    for i in range(1, len(gammas)):
        output_df.loc[f"{column_names[i-1]} to {column_names[i]}", "dropped_themes"] = ", ".join([theme for theme in lists[i-1] if theme not in lists[i]])
        output_df.loc[f"{column_names[i-1]} to {column_names[i]}", "added_themes"] = ", ".join([theme for theme in lists[i] if theme not in lists[i-1]])
        output_df.loc[f"{column_names[i-1]} to {column_names[i]}", "final_themes"] = ", ".join(lists[i])
    return output_df

In [94]:
compare_gamma_for_theme("christmas", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,,NaN,NaN
leidenGamma32.0 to leidenGamma64.0,,,
leidenGamma64.0 to leidenGamma128.0,,,
leidenGamma128.0 to leidenGamma256.0,,,
leidenGamma256.0 to leidenGamma512.0,,,
leidenGamma512.0 to leidenGamma1024.0,,,


In [95]:
compare_gamma_for_theme("Beatles", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"Beatles, Ringo",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Beatles, Ringo",,
leidenGamma64.0 to leidenGamma128.0,"Beatles, Ringo",,
leidenGamma128.0 to leidenGamma256.0,"Beatles, Ringo",,
leidenGamma256.0 to leidenGamma512.0,"Beatles, Ringo",,
leidenGamma512.0 to leidenGamma1024.0,"Beatles, Ringo",,


In [96]:
compare_gamma_for_theme("Family bond", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"Family bonds, Close Relations, Human Connections, Family bond, Family Unity, Family Bonds, Close Relationships, Human Bonds, Family Intertwine",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Family bonds, Close Relations, Human Connections, Family bond, Family Unity, Family Bonds, Close Relationships, Human Bonds, Family Intertwine",,
leidenGamma64.0 to leidenGamma128.0,"Family bonds, Human Connections, Family bond, Family Unity, Family Bonds, Human Bonds, Family Intertwine","Close Relations, Close Relationships",
leidenGamma128.0 to leidenGamma256.0,"Family bonds, Human Connections, Family bond, Family Unity, Family Bonds, Human Bonds, Family Intertwine",,
leidenGamma256.0 to leidenGamma512.0,"Family bonds, Family bond, Family Unity, Family Bonds, Family Intertwine","Human Connections, Human Bonds",
leidenGamma512.0 to leidenGamma1024.0,"Family bond, Family Unity, Family Intertwine","Family bonds, Family Bonds",


In [97]:
compare_gamma_for_theme("dark place", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,,NaN,NaN
leidenGamma32.0 to leidenGamma64.0,,,
leidenGamma64.0 to leidenGamma128.0,,,
leidenGamma128.0 to leidenGamma256.0,,,
leidenGamma256.0 to leidenGamma512.0,,,
leidenGamma512.0 to leidenGamma1024.0,,,


In [98]:
compare_gamma_for_theme("lgbtq, lgbtq+", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"LGBTIQA+, Montreal LGBTQIA2S+, LGBTIQ+, Same-Sex, lgbtq, lgbtq+, lgbtqia, lgbtqia+",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"LGBTIQA+, Montreal LGBTQIA2S+, LGBTIQ+, Same-Sex, lgbtq, lgbtq+, lgbtqia, lgbtqia+",,
leidenGamma64.0 to leidenGamma128.0,"LGBTIQA+, Montreal LGBTQIA2S+, LGBTIQ+, Same-Sex, lgbtq, lgbtq+, lgbtqia, lgbtqia+",,
leidenGamma128.0 to leidenGamma256.0,"LGBTIQA+, Montreal LGBTQIA2S+, LGBTIQ+, Same-Sex, lgbtq, lgbtq+, lgbtqia, lgbtqia+",,
leidenGamma256.0 to leidenGamma512.0,"LGBTIQA+, Montreal LGBTQIA2S+, LGBTIQ+, Same-Sex, lgbtq, lgbtq+, lgbtqia, lgbtqia+",,
leidenGamma512.0 to leidenGamma1024.0,"Montreal LGBTQIA2S+, Same-Sex, lgbtq, lgbtq+, lgbtqia, lgbtqia+","LGBTIQA+, LGBTIQ+",


In [99]:
compare_gamma_for_theme("friendship, friendships", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,,NaN,NaN
leidenGamma32.0 to leidenGamma64.0,,,
leidenGamma64.0 to leidenGamma128.0,,,
leidenGamma128.0 to leidenGamma256.0,,,
leidenGamma256.0 to leidenGamma512.0,,,
leidenGamma512.0 to leidenGamma1024.0,,,


In [100]:
compare_gamma_for_theme("high school friends", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,,NaN,NaN
leidenGamma32.0 to leidenGamma64.0,,,
leidenGamma64.0 to leidenGamma128.0,,,
leidenGamma128.0 to leidenGamma256.0,,,
leidenGamma256.0 to leidenGamma512.0,,,
leidenGamma512.0 to leidenGamma1024.0,,,


In [101]:
compare_gamma_for_theme("musician, musicians", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"Experimental Musician, Budding Musician, Old musician, Famous Dancers, Street Musician, Musician-Dancer, Veteran musician, Sound Engineer, Music Composer, Musical Performance, Young Musician, Solo Performance, Sound artist, Sound Designer, musician, musicians",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Experimental Musician, Budding Musician, Old musician, Famous Dancers, Street Musician, Musician-Dancer, Veteran musician, Music Composer, Musical Performance, Young Musician, Solo Performance, musician, musicians","Sound Engineer, Sound artist, Sound Designer",
leidenGamma64.0 to leidenGamma128.0,"Experimental Musician, Budding Musician, Old musician, Famous Dancers, Street Musician, Musician-Dancer, Veteran musician, Music Composer, Musical Performance, Young Musician, Solo Performance, musician, musicians",,
leidenGamma128.0 to leidenGamma256.0,"Experimental Musician, Budding Musician, Old musician, Famous Dancers, Street Musician, Musician-Dancer, Veteran musician, Music Composer, Musical Performance, Young Musician, Solo Performance, musician, musicians",,
leidenGamma256.0 to leidenGamma512.0,"Experimental Musician, Budding Musician, Old musician, Famous Dancers, Street Musician, Musician-Dancer, Veteran musician, Music Composer, Young Musician, musician, musicians","Musical Performance, Solo Performance",
leidenGamma512.0 to leidenGamma1024.0,"Experimental Musician, Budding Musician, Old musician, Street Musician, Veteran musician, Young Musician, musician, musicians","Famous Dancers, Musician-Dancer, Music Composer",


In [102]:
compare_gamma_for_theme("horse, horses", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"Black Horse, Old Horse, Silver Snow White Horse, horse, horses",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Black Horse, Old Horse, Silver Snow White Horse, horse, horses",,
leidenGamma64.0 to leidenGamma128.0,"Black Horse, Old Horse, Silver Snow White Horse, horse, horses",,
leidenGamma128.0 to leidenGamma256.0,"Black Horse, Old Horse, Silver Snow White Horse, horse, horses",,
leidenGamma256.0 to leidenGamma512.0,"Black Horse, Old Horse, Silver Snow White Horse, horse, horses",,
leidenGamma512.0 to leidenGamma1024.0,"Black Horse, Old Horse, Silver Snow White Horse, horse, horses",,


# Write best Leiden communities to Neo4j
Based on the examples above, 512.0 seems like a good level for gamma.
Rename the property LeidenGamma32.0 from the graph projection to leidenId in the on-disk graph.
Drop any leidenId values from a previous run.

In [103]:
gds.run_cypher("MATCH (t:Theme|Stem) SET t.leidenId = null")
gds.graph.nodeProperties.write(g_themes2, {"leidenGamma512.0": "leidenId"})

writeMillis                      254
graphName            reweight_themes
nodeProperties            [leidenId]
propertiesWritten              14998
Name: 0, dtype: object

Create ThemeGroup nodes connecting themes that are duplicates.

In [104]:
gds.run_cypher("""CREATE CONSTRAINT theme_group_node_key IF NOT EXISTS FOR (n:ThemeGroup) REQUIRE n.id IS NODE KEY""")

""


In [105]:
gds.run_cypher("""
MATCH (t:Theme)
WHERE t.leidenId IS NOT NULL
WITH t ORDER BY t.description
WITH t.leidenId AS id, collect(t) AS themeNodes, collect(t.description) AS descriptions
MERGE (g:ThemeGroup {id:id}) 
SET g.descriptions = descriptions
WITH g, themeNodes
FOREACH (t in themeNodes | MERGE (t)-[:IN_GROUP]->(g))
""")

""


In [106]:
gds.run_cypher("""
MATCH (s:Stem)
WHERE s.leidenId IS NOT NULL
CALL
{
WITH s
MERGE (g:ThemeGroup {id:s.leidenId}) 
SET g.descriptions = coalesce(g.descriptions, []) + s.descriptions
WITH s, g
MATCH (s)<-[:HAS_STEM]-(t)
MERGE (t)-[:IN_GROUP]->(g)
}
""")

""


Create ThemeGroup nodes for Theme nodes that are not already grouped in a theme group if the theme is referenced in more than one document.

In [107]:
gds.run_cypher("""
MATCH (g:ThemeGroup)
WITH max(g.id) AS maxId
MATCH (t:Theme&Groupable)
WHERE NOT EXISTS {(t)-[:IN_GROUP]->()}
WITH maxId, COLLECT(t) AS needsGroup
UNWIND range(0, size(needsGroup)-1) AS idx
WITH maxId, idx, needsGroup[idx] AS t
MERGE (g:ThemeGroup {id:maxId + idx + 1})
SET g.descriptions = [t.description]
WITH g, t
MERGE (t)-[:IN_GROUP]->(g)
RETURN COUNT(*) AS singleThemeGroups
""")

,singleThemeGroups
0,34347


In [108]:
gds.run_cypher("""
MATCH (g:ThemeGroup)
WITH max(g.id) AS maxId
MATCH (s:Stem)
WHERE s.leidenId IS NULL
WITH maxId, COLLECT(s) AS needsGroup
UNWIND range(0, size(needsGroup)-1) AS idx
WITH maxId, idx, needsGroup[idx] AS s
CALL
{
WITH s, maxId, idx
MERGE (g:ThemeGroup {id:maxId + idx + 1}) 
SET g.descriptions = s.descriptions
WITH s, g
MATCH (s)<-[:HAS_STEM]-(t)
MERGE (t)-[:IN_GROUP]->(g)
}
""")

""


For Theme nodes that are not already in a theme group and contain only one document, create ThemeGroup nodes and add a simple summary.

In [109]:
single_doc_df = gds.run_cypher("""
    MATCH (g:ThemeGroup)
    WITH max(g.id) AS maxId
    MATCH (t:Theme) WHERE NOT EXISTS {(t)-[:IN_GROUP]->()} 
    WITH maxId, COLLECT(t) AS needsGroup
    UNWIND range(0, size(needsGroup)-1) AS idx
    WITH maxId, idx, needsGroup[idx] AS t
    CALL
    {
    WITH t, maxId, idx
    MERGE (g:ThemeGroup {id:maxId + idx + 1})
    SET g.descriptions = [t.description]
    SET g.summary = "A movie about " + t.description
    MERGE (t)-[:IN_GROUP]->(g)
    RETURN g
    } IN TRANSACTIONS OF 10000 ROWS
    RETURN g.id AS groupId, g.summary AS summary""")

In [116]:
single_doc_df.head()

,groupId,summary


In [113]:
! pip install sentence-transformers --quiet

In [114]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

In [115]:
single_doc_df['embedding'] = embeddings.embed_documents(single_doc_df['summary'].tolist())

IndexError: list index out of range

In [ ]:
gds.run_cypher("""
UNWIND $data AS row
CALL
{
WITH row
MATCH (g:ThemeGroup {id:row['groupId']})
CALL db.create.setNodeVectorProperty(g, 'summaryEmbedding', row['embedding'])
} IN TRANSACTIONS OF 10000 ROWS
""", {"data": single_doc_df[['groupId', 'embedding']].to_dict("records")})

## Test simple summaries for other groups

In [97]:
needs_summary_df = gds.run_cypher("""
MATCH (g:ThemeGroup) WHERE g.summary IS NULL
SET g.summary = 
    CASE WHEN size(g.descriptions) = 1 THEN "Movies about " + g.descriptions[0]
    ELSE reduce(s = "Movies about " + g.descriptions[0], d in g.descriptions[1..-1] | s + ", " + d) + ", and " + g.descriptions[-1]
    END
RETURN g.id AS groupId, g.summary AS summary""")

In [100]:
needs_summary_df['embedding'] = embeddings.embed_documents(needs_summary_df['summary'].tolist())

In [101]:
gds.run_cypher("""
UNWIND $data AS row
CALL
{
WITH row
MATCH (g:ThemeGroup {id:row['groupId']})
CALL db.create.setNodeVectorProperty(g, 'summaryEmbedding', row['embedding'])
} IN TRANSACTIONS OF 10000 ROWS
""", {"data": needs_summary_df[['groupId', 'embedding']].to_dict("records")})

""


# Clean up projections

In [102]:
gds.graph.drop(g_themes)

graphName                                                                                                                                                                                                                                                                                                                                                                                                                        themes
database                                                                                                                                                                                                                                                                                                                                                                                                                          neo4j
databaseLocation                                                                                                                                        

In [103]:
gds.graph.drop(g_themes2)

graphName                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [104]:
gds.run_cypher("MATCH (g:Groupable) REMOVE g:Groupable")

""


In [105]:
gds.run_cypher("DROP INDEX groupbale_vectors")

""


# Add mean vector representation to each ThemeGroup

In [106]:
group_embeddings_df = gds.run_cypher("""
MATCH (g:ThemeGroup)<-[:IN_GROUP]-(t)
RETURN g.id AS groupId, collect(t.embedding) AS embeddings
""")

In [107]:
group_embeddings_df['mean_embedding'] = group_embeddings_df.apply(get_mean_vector, axis=1)

In [108]:
gds.run_cypher("""
    UNWIND $data AS row
    CALL
    {
    WITH row
    MATCH (g:ThemeGroup {id: row['groupId']})
    CALL db.create.setNodeVectorProperty(g, "meanEmbedding", row['mean_embedding'])
    } IN TRANSACTIONS OF 10000 ROWS""",
                   {"data":group_embeddings_df[['groupId', 'mean_embedding']].to_dict("records")})


""


In [109]:
gds.run_cypher("""CREATE VECTOR INDEX theme_group_mean_vectors IF NOT EXISTS 
                  FOR (t:ThemeGroup)
                  ON (t.meanEmbedding)
                  OPTIONS {indexConfig: 
                      {`vector.dimensions`: 1536,
                       `vector.similarity_function`: 'cosine'
                       }}
                       """)

""
